In [16]:
# 필요한 패키지 설치
import requests
from bs4 import BeautifulSoup
import io
import zipfile
import xmltodict
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np

## 고유번호 개발 가이드
1. 요청 파라미터
- 고유번호 corp_code 
- 정식명칭 corp_name
- 종목코드 stock_code
- 최종변경일자 modify_date

2. 추출 결과값
- 

In [17]:
# https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018
KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'

url = '	https://opendart.fss.or.kr/api/corpCode.xml'
params = {'crtfc_key' : KEY}

response = requests.get(url, params = params).content


# 압축 파일 풀어서 xml파일 디렉토리에 저장하기
with ZipFile(BytesIO(response)) as zipfile:
    zipfile.extractall('corpCode')

#corpcode xml 파일 생김

In [18]:
# xml 파일 읽어오기, 여기서 고유번호와 종목명이 포함된 리스트를 만들 것임
xmlTree = parse('./corpCode/CORPCODE.xml')
root = xmlTree.getroot()
raw_list = root.findall('list')

corp_list = []

for l in range(len(raw_list)):
    corp_code = raw_list[l].findtext('corp_code')
    corp_name = raw_list[l].findtext('corp_name')
    stock_code = raw_list[l].findtext('stock_code')
    modify_date = raw_list[l].findtext('modify_date')
    
    corp_list.append([corp_code, corp_name, stock_code, modify_date])
    
#corp_list 형성
# corp_list

#corp_list -> corp_df


from pandas import DataFrame
from datetime import datetime

#위에서 생성한 corp_list에 라벨링하여 데이터프레임화

corp_df = DataFrame(corp_list, columns=[
    '고유번호',
    '정식명칭', 
    '종목코드', 
    '최종변경일자'
])

#corp_df 의 결측치 drop시킨 stock_df (종목코드가 없는 종목을 drop하고, 최종변경일자는 쓰지 않으므로 칼럼 전체를 drop)

stock_df = corp_df[corp_df['종목코드'] != " "]
stock_df = stock_df[['고유번호', '정식명칭', '종목코드', '최종변경일자']].dropna()
# stock_df

In [19]:
# stock_df에서 고유번호를 호출해오는 함수 형성
def goyu_searcher(COMPANYNAME):
    temp_df = stock_df.loc[(stock_df['정식명칭'] == COMPANYNAME)]
    return temp_df.iloc[0, 0]

In [20]:
# stock_df에서 종목 이름을 호출해오는 함수 형성
def corpname_searcher(CORP_CODE):
    temp_df = stock_df.loc[(stock_df['고유번호'] == CORP_CODE)]
    return temp_df.iloc[0, 1]

In [21]:
#######

crno_list = []

corp_list_temp = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
for corp in corp_list_temp:
    crno_list.append(goyu_searcher(corp))

crno_list

['00126380', '00126256', '00164742', '00413046', '00155319']

In [22]:
# 가져온 corp_code를 바탕으로 

def get_items(KEY, CORP_CODE, YEAR, RPT_CODE, FS_DIV):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from lxml import html
    from urllib.request import Request, urlopen
    from urllib.parse import urlencode, quote_plus, unquote
    
    
    url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
    params={'crtfc_key' : KEY,
            'corp_code' : CORP_CODE, 
            'bsns_year' : YEAR, 
            'reprt_code' : RPT_CODE, 
            'fs_div' : FS_DIV}
    
    response = requests.get(url, params = params)
    
    import json
    try:
            json_obj = json.loads(response.text)["list"]
    except:
        #     json_obj = ["## ERROR", corpname_searcher(CORP_CODE), YEAR, RPT_CODE]
            json_obj = ["## ERROR", corpname_searcher(CORP_CODE), YEAR] #ok
            # json_obj = [corpname_searcher(CORP_CODE), RPT_CODE] ok
        #     json_obj = [corpname_searcher(CORP_CODE), YEAR, RPT_CODE]
        
    
    return json_obj

In [23]:
#get_items()를 사용하여 구하고자 하는 기업들의 재무제표 정보 가져오기

corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)
reprtlist = ['11013', '11012', '11014', '11011']
fs_divs = ['CFS', 'OFS']

result_list = []

for corp in corp_list:
    for year in year_list:
        for reprt in reprtlist:
            for fs_div in fs_divs:
                result_list.extend(get_items(KEY, goyu_searcher(corp), year, reprt, fs_div))
                
result_list
            

[{'rcept_no': '20180515001699',
  'reprt_code': '11013',
  'bsns_year': '2018',
  'corp_code': '00126380',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'account_id': 'ifrs_CurrentAssets',
  'account_nm': '유동자산',
  'account_detail': '-',
  'thstrm_nm': '제 50 기 1분기말',
  'thstrm_amount': '154941953000000',
  'frmtrm_nm': '제 49 기말',
  'frmtrm_amount': '146982464000000',
  'ord': '1',
  'currency': 'KRW'},
 {'rcept_no': '20180515001699',
  'reprt_code': '11013',
  'bsns_year': '2018',
  'corp_code': '00126380',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'account_id': 'ifrs_CashAndCashEquivalents',
  'account_nm': '현금및현금성자산',
  'account_detail': '-',
  'thstrm_nm': '제 50 기 1분기말',
  'thstrm_amount': '32303752000000',
  'frmtrm_nm': '제 49 기말',
  'frmtrm_amount': '30545130000000',
  'ord': '2',
  'currency': 'KRW'},
 {'rcept_no': '20180515001699',
  'reprt_code': '11013',
  'bsns_year': '2018',
  'corp_code': '00126380',
  'sj_div': 'BS',
  'sj_nm': '재무상태표',
  'account_id': 'dart_ShortTermDepositsNotCl

In [24]:
# 구한 결과값을 df로 변환


#틀을 먼저 만들고
rcept_no_dfcol = []
corp_name_dfcol = []
corp_code_dfcol = []
bsns_year_dfcol = []
reprt_code_dfcol = []

result_list = [dict_row for dict_row in result_list if isinstance(dict_row, dict)]

for dict_row in result_list:
    rcept_no_dfcol.append(dict_row['rcept_no'])
    corp_name_dfcol.append(corpname_searcher(dict_row['corp_code']))
    corp_code_dfcol.append(dict_row['corp_code'])
    bsns_year_dfcol.append(dict_row['bsns_year'])
    reprt_code_dfcol.append(dict_row['reprt_code'])

data = {'기준일자': rcept_no_dfcol, '기업명': corp_name_dfcol, '종목코드': corp_code_dfcol, '사업년도': bsns_year_dfcol, '분기코드': reprt_code_dfcol}

result_df = pd.DataFrame(data = data)
result_df

,기준일자,기업명,종목코드,사업년도,분기코드
0,20180515001699,삼성전자,00126380,2018,11013
1,20180515001699,삼성전자,00126380,2018,11013
2,20180515001699,삼성전자,00126380,2018,11013
3,20180515001699,삼성전자,00126380,2018,11013
4,20180515001699,삼성전자,00126380,2018,11013
...,...,...,...,...,...
28550,20230515001841,POSCO홀딩스,00155319,2023,11013
28551,20230515001841,POSCO홀딩스,00155319,2023,11013
28552,20230515001841,POSCO홀딩스,00155319,2023,11013
28553,20230515001841,POSCO홀딩스,00155319,2023,11013


In [25]:
# 지표 데이터를 넣을 컬럼 리스트 생성

data_dfcol = [] # 바로 데이터프레임에 컬럼 추가할거긴 한데, 일단 어떤 내용이 들어간지도 모이면 좋을 것 같아서

for dict_row in result_list:
    data_dfcol.append(dict_row['account_nm'])
data_dfcol = list(set(data_dfcol))

for data_col in data_dfcol:
    result_df[data_col] = None
    

result_df = result_df.drop_duplicates().reset_index(drop=True)
result_df

/var/folders/h4/90vtzp5s1s3b0vhwb76zfx900000gn/T/ipykernel_27582/1002845127.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[data_col] = None


,기준일자,기업명,종목코드,사업년도,분기코드,기말의 현금및현금성자산,후속적으로 당기손익으로 재분류되지 않는 항목,상환전환우선주 상환,단기차입금의 순차입,해외사업환산손익,...,상각후원가금융자산의 취득,관계기업 및 공동기업 투자의 취득,재무활동으로 인한 현금유입액,자본총계,유형자산의 처분,현금흐름위험회피 파생상품평가손익,기타영업외비용,정부보조금의 수령,연결반기순이익,장기충당부채
0,20180515001699,삼성전자,00126380,2018,11013,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,20180814001113,삼성전자,00126380,2018,11012,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,20181114001530,삼성전자,00126380,2018,11014,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,20190401004781,삼성전자,00126380,2018,11011,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,20190515001605,삼성전자,00126380,2019,11013,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20220516002202,POSCO홀딩스,00155319,2022,11013,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
80,20220816000612,POSCO홀딩스,00155319,2022,11012,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
81,20221114002122,POSCO홀딩스,00155319,2022,11014,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
82,20230309000896,POSCO홀딩스,00155319,2022,11011,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [26]:
for dict_row in result_list:
    # 종목코드, 사업년도, 분기코드가 같은 행을 df에서 찾아서 숫자를 탐색, df에 저장
    
    corp_code = dict_row['corp_code']
    bsns_year = dict_row['bsns_year']
    reprt_code = dict_row['reprt_code']
    
    acc_name = dict_row['account_nm']
    acc_data = dict_row['thstrm_amount']
    
    condition = (result_df['종목코드'] == corp_code) & (result_df['사업년도'] == bsns_year) & (result_df['분기코드'] == reprt_code)
    
    result_df.at[result_df.loc[condition].index, acc_name] = acc_data


print(result_df)



              기준일자       기업명      종목코드  사업년도   분기코드 기말의 현금및현금성자산  \
0   20180515001699      삼성전자  00126380  2018  11013         None   
1   20180814001113      삼성전자  00126380  2018  11012         None   
2   20181114001530      삼성전자  00126380  2018  11014         None   
3   20190401004781      삼성전자  00126380  2018  11011         None   
4   20190515001605      삼성전자  00126380  2019  11013         None   
..             ...       ...       ...   ...    ...          ...   
79  20220516002202  POSCO홀딩스  00155319  2022  11013         None   
80  20220816000612  POSCO홀딩스  00155319  2022  11012         None   
81  20221114002122  POSCO홀딩스  00155319  2022  11014         None   
82  20230309000896  POSCO홀딩스  00155319  2022  11011         None   
83  20230515001841  POSCO홀딩스  00155319  2023  11013         None   

   후속적으로 당기손익으로 재분류되지 않는 항목 상환전환우선주 상환     단기차입금의 순차입      해외사업환산손익  ...  \
0                      None       None           None          None  ...   
1                      None    

In [27]:
dict_row = result_list[0]


corp_code = dict_row['corp_code']
bsns_year = dict_row['bsns_year']
reprt_code = dict_row['reprt_code']

acc_name = dict_row['account_nm']
acc_data = dict_row['thstrm_amount']



In [28]:
test_result_df = result_df.dropna(axis='columns')

# '분기명' 열 생성
test_result_df['분기명'] = test_result_df['분기코드'].map({'11013': '1분기', '11012': '2분기', '11014': '3분기', '11011': '4분기'})
cols = test_result_df.columns.tolist()  # 열 인덱스 리스트로 변환
cols[3], cols[26] = cols[26], cols[3]  # 열 위치 교환
test_result_df = test_result_df[cols]
test_result_df

/var/folders/h4/90vtzp5s1s3b0vhwb76zfx900000gn/T/ipykernel_27582/2273132327.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_result_df['분기명'] = test_result_df['분기코드'].map({'11013': '1분기', '11012': '2분기', '11014': '3분기', '11011': '4분기'})


,기준일자,기업명,종목코드,유형자산의 처분,분기코드,이자의 수취,비유동자산,재고자산,매출총이익,비지배지분,...,유형자산,금융수익,매출원가,유형자산의 취득,무형자산,자산총계,유동자산,무형자산의 취득,사업년도,분기명
0,20180515001699,삼성전자,00126380,141569000000,11013,94118000000,132836079000000,8915860000000,17410718000000,94546000000,...,66737192000000,799826000000,25196260000000,7870910000000,2823356000000,205206164000000,72370085000000,218113000000,2018,1분기
1,20180814001113,삼성전자,00126380,198281000000,11012,222063000000,135899888000000,9935909000000,16708550000000,81090000000,...,69235302000000,1290033000000,24606173000000,13744369000000,2857695000000,203970112000000,68070224000000,429125000000,2018,2분기
2,20181114001530,삼성전자,00126380,282945000000,11014,360840000000,135895097000000,10357340000000,20067142000000,178867000000,...,69525464000000,739900000000,27578197000000,17692885000000,2814997000000,217799444000000,81904347000000,604961000000,2018,3분기
3,20190401004781,삼성전자,00126380,340558000000,11011,459074000000,138981902000000,12440951000000,68715364000000,450142000000,...,70602493000000,3737494000000,101666506000000,21387378000000,2901476000000,219021357000000,80039455000000,880032000000,2018,4분기
4,20190515001605,삼성전자,00126380,105460000000,11013,98290000000,138335268000000,14336408000000,10586172000000,31372000000,...,70498616000000,643068000000,26452223000000,2821582000000,2876176000000,220646957000000,82311689000000,250866000000,2019,1분기
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20220516002202,POSCO홀딩스,00155319,-14485002642,11013,38674284562,47458142280986,,2928765425577,214947479400,...,143758772501,-125555822774,6465259462743,355821595282,13676722585,53333865909937,5875723628951,8668278674,2022,1분기
80,20220816000612,POSCO홀딩스,00155319,-14485002642,11012,62434274729,47422658774188,,2733139872051,254454407727,...,143189169662,-258483606915,-190076440,355821595282,13513844769,51483262433593,4060603659405,8668278674,2022,2분기
81,20221114002122,POSCO홀딩스,00155319,-14485002642,11014,87251112870,47402753865253,,1607959638506,213864027182,...,142646703013,-359096307742,-950323950,356941976458,14401244987,50860490239671,3457736374418,9519018576,2022,3분기
82,20230309000896,POSCO홀딩스,00155319,-14447214917,11011,44614715351,46840829094141,,7649292423855,413709215105,...,145005572040,-346941713822,6463245690193,359532593243,15901608719,51149119209287,4308290115146,10194840491,2022,4분기


In [29]:
stock_codes = {'삼성전자' : '005930', '셀트리온' : '068270', '삼성생명' : '032830', 'POSCO홀딩스' : '005490', '현대자동차' : '005380'}

test_result_df['주가코드'] = test_result_df.apply(lambda row: stock_codes.get(row.기업명), axis=1)
test_result_df



,기준일자,기업명,종목코드,유형자산의 처분,분기코드,이자의 수취,비유동자산,재고자산,매출총이익,비지배지분,...,금융수익,매출원가,유형자산의 취득,무형자산,자산총계,유동자산,무형자산의 취득,사업년도,분기명,주가코드
0,20180515001699,삼성전자,00126380,141569000000,11013,94118000000,132836079000000,8915860000000,17410718000000,94546000000,...,799826000000,25196260000000,7870910000000,2823356000000,205206164000000,72370085000000,218113000000,2018,1분기,005930
1,20180814001113,삼성전자,00126380,198281000000,11012,222063000000,135899888000000,9935909000000,16708550000000,81090000000,...,1290033000000,24606173000000,13744369000000,2857695000000,203970112000000,68070224000000,429125000000,2018,2분기,005930
2,20181114001530,삼성전자,00126380,282945000000,11014,360840000000,135895097000000,10357340000000,20067142000000,178867000000,...,739900000000,27578197000000,17692885000000,2814997000000,217799444000000,81904347000000,604961000000,2018,3분기,005930
3,20190401004781,삼성전자,00126380,340558000000,11011,459074000000,138981902000000,12440951000000,68715364000000,450142000000,...,3737494000000,101666506000000,21387378000000,2901476000000,219021357000000,80039455000000,880032000000,2018,4분기,005930
4,20190515001605,삼성전자,00126380,105460000000,11013,98290000000,138335268000000,14336408000000,10586172000000,31372000000,...,643068000000,26452223000000,2821582000000,2876176000000,220646957000000,82311689000000,250866000000,2019,1분기,005930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,20220516002202,POSCO홀딩스,00155319,-14485002642,11013,38674284562,47458142280986,,2928765425577,214947479400,...,-125555822774,6465259462743,355821595282,13676722585,53333865909937,5875723628951,8668278674,2022,1분기,005490
80,20220816000612,POSCO홀딩스,00155319,-14485002642,11012,62434274729,47422658774188,,2733139872051,254454407727,...,-258483606915,-190076440,355821595282,13513844769,51483262433593,4060603659405,8668278674,2022,2분기,005490
81,20221114002122,POSCO홀딩스,00155319,-14485002642,11014,87251112870,47402753865253,,1607959638506,213864027182,...,-359096307742,-950323950,356941976458,14401244987,50860490239671,3457736374418,9519018576,2022,3분기,005490
82,20230309000896,POSCO홀딩스,00155319,-14447214917,11011,44614715351,46840829094141,,7649292423855,413709215105,...,-346941713822,6463245690193,359532593243,15901608719,51149119209287,4308290115146,10194840491,2022,4분기,005490


In [30]:
test_result_df.to_csv('api_final_result.csv', index=False)